In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pydataset import data
import env
from env import get_db_url
# train test split from sklearn
from sklearn.model_selection import train_test_split
# imputer from sklearn
from sklearn.impute import SimpleImputer

# filter out warnings
import warnings
warnings.filterwarnings('ignore')

# our own acquire script:
import acquire 

1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

You should end up with something like this:

In [2]:
# Importing our data


attendance = pd.read_sql('SELECT * FROM attendance', get_db_url('tidy_data'))

attendance.head()


,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
attendance = attendance.rename(columns={"Unnamed: 0": "students"})
attendance

,students,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [4]:
attendance = attendance.melt(id_vars=['students'], var_name='date', value_name='status')
attendance

,students,date,status
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


In [5]:
#Ryans's example
#attendance.status.map({'A': 0, 'P': 1, 'H': .5, 'T': .9})

In [6]:
attendance['status']= attendance['status'].map({'A': 0, 'P': 1, 'H': .5, 'T': .9})
attendance

,students,date,status
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9
5,Jane,2018-01-02,1.0
6,Billy,2018-01-02,0.9
7,John,2018-01-02,0.9
8,Sally,2018-01-03,0.9
9,Jane,2018-01-03,0.9


In [7]:
attendance.groupby('students').mean()

,status
students,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


2. Coffee Levels

Read the coffee_levels table.

Transform the data so that each carafe is in it's own column.

Is this the best shape for the data?


In [8]:
coffee = pd.read_sql('SELECT * FROM coffee_levels', get_db_url('tidy_data'))

coffee.head()


,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [9]:
coffee.pivot_table(index='hour', columns='coffee_carafe')

coffee_amount                    
coffee_carafe             x         y         z
hour                                           
8                  0.816164  0.189297  0.999264
9                  0.451018  0.521502  0.915990
10                 0.843279  0.023163  0.144928
11                 0.335533  0.235529  0.311495
12                 0.898291  0.017009  0.771947
13                 0.310711  0.997464  0.398520
14                 0.507288  0.058361  0.864464
15                 0.215043  0.144644  0.436364
16                 0.183891  0.544676  0.280621
17                 0.391560  0.594126  0.436677

3.Cake Recipes

Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

Tidy the data as necessary.

Which recipe, on average, is the best? recipe b

Which oven temperature, on average, produces the best results? 275

Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [11]:
cake = pd.read_sql('SELECT * FROM cake_recipes', get_db_url('tidy_data'))

cake


,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [12]:
cake= cake.melt(id_vars='recipe:position', var_name='temp', value_name='score')
cake

,recipe:position,temp,score
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207
5,c:top,225,71.306308
6,d:bottom,225,52.799753
7,d:top,225,96.873178
8,a:bottom,250,53.912627
9,a:top,250,52.009735


In [13]:
cake[['recipe', 'position']] = cake['recipe:position'].str.split(':', expand=True)

In [14]:
cake

,recipe:position,temp,score,recipe,position
0,a:bottom,225,61.738655,a,bottom
1,a:top,225,51.709751,a,top
2,b:bottom,225,57.095320,b,bottom
3,b:top,225,82.455004,b,top
4,c:bottom,225,96.470207,c,bottom
5,c:top,225,71.306308,c,top
6,d:bottom,225,52.799753,d,bottom
7,d:top,225,96.873178,d,top
8,a:bottom,250,53.912627,a,bottom
9,a:top,250,52.009735,a,top


In [15]:
cake = cake.drop(columns='recipe:position')
cake

,temp,score,recipe,position
0,225,61.738655,a,bottom
1,225,51.709751,a,top
2,225,57.095320,b,bottom
3,225,82.455004,b,top
4,225,96.470207,c,bottom
5,225,71.306308,c,top
6,225,52.799753,d,bottom
7,225,96.873178,d,top
8,250,53.912627,a,bottom
9,250,52.009735,a,top


In [16]:
cake.groupby('recipe').mean().idxmax()

score    b
dtype: object

In [17]:
cake.groupby('recipe').mean().max()

score    76.736074
dtype: float64

In [18]:
cake

,temp,score,recipe,position
0,225,61.738655,a,bottom
1,225,51.709751,a,top
2,225,57.095320,b,bottom
3,225,82.455004,b,top
4,225,96.470207,c,bottom
5,225,71.306308,c,top
6,225,52.799753,d,bottom
7,225,96.873178,d,top
8,250,53.912627,a,bottom
9,250,52.009735,a,top


In [19]:
cake.groupby('temp').mean().sort_values('score').tail(1)

,score
temp,
275,74.886754


In [20]:
cake.groupby('temp').mean().max()

score    74.886754
dtype: float64

In [21]:
cake.groupby(['recipe', 'position', 'temp']).mean().idmax()

AttributeError: 'DataFrame' object has no attribute 'idmax'

In [22]:
cake.groupby(['recipe', 'position', 'temp']).mean().max()

score    99.248541
dtype: float64

In [ ]:
cake.groupby(['recipe', 'position', 'temp']).mean().sort_values('score').tail(1)

4. Bonus: 
Explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.